In [63]:
import pandas as pd
from statsmodels.stats.proportion import proportions_ztest

# Archivo separado por comas
df_webdata = pd.read_csv('df_webdata.csv')
df_final = pd.read_csv('df_final_clients.csv')

# Filtrar df_webdata para crear df_participants:
df_participants = df_webdata[df_webdata['variation'] != "Not participant"]

# Verificar las primeras filas del nuevo DataFrame
df_participants.head()

,client_id,visit_id,duration,start,step_1,step_2,step_3,confirm,variation
3,555,637149525_38041617439_716659,00:02:38,1,1,1,1,1,Test
4,647,40369564_40101682850_311847,00:06:17,1,1,1,1,1,Test
11,934,7076463_57954418406_971348,00:02:22,4,0,0,0,0,Test
12,1028,557292053_87239438319_391157,00:08:58,1,5,2,1,0,Control
14,1104,543158812_46395476577_767725,00:00:00,1,0,0,0,0,Control


In [64]:
#Hipótesis Nula (H0): No hay diferencia en el completion rate entre los grupos.
#Hipótesis Alternativa (H1): Hay una diferencia significativa en el completion rate entre los grupos.

In [65]:
def paso_completado(val):
    return 1 if val > 0 else 0

# Crear la nueva columna 'completion' sumando los pasos binarizados
#df_participants.loc[:, 'completion'] = (
#    df_participants['start'].apply(paso_completado) +
#    df_participants['step_1'].apply(paso_completado) +
#    df_participants['step_2'].apply(paso_completado) +
#    df_participants['step_3'].apply(paso_completado) +
#    df_participants['confirm'].apply(paso_completado) )

df_participants.loc[:, 'completion'] = (df_participants['confirm'].apply(paso_completado))

# Ahora, para calcular el completion rate, filtramos los que completaron los 5 pasos
completion_rate_control = df_participants[(df_participants['variation'] == 'Control') & (df_participants['completion'] == 1)].shape[0] / df_participants[df_participants['variation'] == 'Control'].shape[0] * 100
completion_rate_test = df_participants[(df_participants['variation'] == 'Test') & (df_participants['completion'] == 1)].shape[0] / df_participants[df_participants['variation'] == 'Test'].shape[0] * 100

print(f'Completion Rate Grupo Control: {completion_rate_control}%')
print(f'Completion Rate Grupo Test: {completion_rate_test}%')

Completion Rate Grupo Control: 49.89610147939088%
Completion Rate Grupo Test: 58.571658961401994%


C:\Users\Usuario\AppData\Local\Temp\ipykernel_18056\3791403091.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_participants.loc[:, 'completion'] = (df_participants['confirm'].apply(paso_completado))


In [66]:
# Calcular el número de éxitos (aquellos que completaron los 5 pasos) y el total de cada grupo
exitos_control = df_participants[(df_participants['variation'] == 'Control') & (df_participants['completion'] == 1)].shape[0]
total_control = df_participants[df_participants['variation'] == 'Control'].shape[0]

exitos_test = df_participants[(df_participants['variation'] == 'Test') & (df_participants['completion'] == 1)].shape[0]
total_test = df_participants[df_participants['variation'] == 'Test'].shape[0]

In [67]:


# Lista de éxitos y totales
exitos = [exitos_control, exitos_test]
totales = [total_control, total_test]

# Prueba de proporciones
stat, p_value = proportions_ztest(exitos, totales)

# Mostrar los resultados
print(f'Estadístico z: {stat:.4f}')
print(f'Valor p: {p_value:.4f}')

# El estadístico z indica la distancia, en unidades de desviación estándar, 
# de la diferencia observada respecto a la hipotesis nula de que no hay diferencia.
# Como sale muy grande en valores absolutos, indica que la diferencia en los 
# completion rates es mucho mayor de lo que esperaríamos si no hubiera diferencia real.

# El valor p te dice la probabilidad de observar la diferencia si la hipótesis nula 
# fuera verdadera. Como es muy pequeño, se puede concluir que hay una diferencia 
# estadísticamente significativa.

# El resultado es estadísticamente significativo y además la diferencia en 
# completion rate es más del 5%, podemos afirmar con mayor confianza que las 
# modificaciones realizadas en tu experimento tuvieron un impacto positivo.

Estadístico z: -22.8989
Valor p: 0.0000


In [68]:
#Hipótesis Nula (H0): No hay diferencia en la duracion entre los grupos.
#Hipótesis Alternativa (H1): Hay una diferencia significativa en la duracion entre los grupos.

In [70]:
# Filtrar los datos para solo aquellos con completion == 1
completed_sessions = df_participants[df_participants['completion'] == 1]

# Comprobar tipos después de la conversión
print(completed_sessions['duration'].dtype)

completed_sessions['duration']

object


3         00:02:38
4         00:06:17
18        00:04:05
19        00:01:35
21        00:03:31
            ...   
159093    00:02:45
159096    00:03:17
159098    00:02:51
159101    00:01:59
159106    00:01:15
Name: duration, Length: 37879, dtype: object

In [62]:
# Definir una función para convertir hh:mm:ss a segundos
def time_to_seconds(tiempo):
    try:
        h, m, s = map(int, tiempo.split(':'))
        return h * 3600 + m * 60 + s
    except:
        return None  # Devuelve None en caso de error de conversión

# Aplicar la función a la columna 'duration'
completed_sessions['duration_seconds'] = completed_sessions['duration'].apply(time_to_seconds)

# Comprobar si ha funcionado la conversión
print(completed_sessions['duration_seconds'].head())

3     None
4     None
18    None
19    None
21    None
Name: duration_seconds, dtype: object


C:\Users\Usuario\AppData\Local\Temp\ipykernel_18056\2956164803.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  completed_sessions['duration_seconds'] = completed_sessions['duration'].apply(time_to_seconds)


In [52]:
# Convertir las duraciones a segundos
completed_sessions.loc[:, 'duration_seconds'] = completed_sessions['duration'].dt.total_seconds()

AttributeError: Can only use .dt accessor with datetimelike values

In [ ]:


# Separar las duraciones de las sesiones por grupo
duraciones_control = completed_sessions[completed_sessions['variation'] == 'Control']['duration']
duraciones_test = completed_sessions[completed_sessions['variation'] == 'Test']['duration']

from scipy.stats import ttest_ind

# Realizar la prueba t
t_stat, p_value = ttest_ind(duraciones_control, duraciones_test, equal_var=False) # Usar equal_var=False si no asumimos varianzas iguales

# Mostrar los resultados
print(f'Estadístico t: {t_stat:.4f}')
print(f'Valor p: {p_value:.4f}')